## Homework_2

Для выполнения данной лабораторной работы по Python в базе данных 'derebot' в cхеме 'SAMCHUK_SV' были созданы 2 таблицы:
- 'costs' - для хранения трат, разделенных по группам
- 'costs_limits' - для хранения лимитов трат

Произведем обработку и вставку сырых данных в таблицу 'costs' для проверки работы методов.

In [18]:
from config_db import config_db
from database import DataBase


db = DataBase(**config_db)

db.insert_tbl_costs(
    [
        ['12.10.2021', 'Канцелярия', 300],
        ['12-10-2021', '///Кофе///', 123.45],
        [' 12 .10.2021 ', '|||Кофе', 99.99],
        ['12.10/2021', 'БеНзИн/,', 642],
        ['12/10/2021', '    КОФЕ....   ', 100],
        ['13.10.2021', '????КанцеЛярия?', 200],
        ['13.10.2021', 'кофЕ ', 432.10],
        ['13.10.2021', '  беНзин', 100],
        ['14_10.2021', '--кАНЦЕЛЯРИя---', 500],
        ['14_10.2021', '$$кофе$$ ', 50],
        ['  14 - 10 - 2021', '  %%кАнЦеЛяРия:!@', 23.09],
    ]
)

#### Задание 1
<p style="font-size: 25px">&#10077;</p>
<i>БД = Oracle
<br>Используйте sqlalchemy

Необходимо создать функции расчета трат

<br>Первая функция должна рассчитывать траты по всем группам в рамках одного дня.
<br>То есть, вы передаёте дату, а функция возвращает вам агрегированные суммы трат по группам
<br>Например: у нас имеется три записи: 
<br>20.01.2021 Продукты 100
<br>20.01.2021 Продукты 200
<br>19.01.2021 Такси 150
<br>Если передать функции дату 20.01.2021, то она должна вывести: 
<br>20.01.2021 Продукты 300</i>
<p style="font-size: 25px">&#10078;</p>

#### Решение задания 1

Воспользуемся методом calc_costs класса DataBase для рассчета трат по группам в рамках одного дня:
```python
def calc_costs(self, date: str):
    try:
        self.__metadata.reflect()
        tbl = self.__metadata.tables['costs']
        date = self.__check_date(date)

        transaction = self.conn.begin()
        responce = select(
            tbl.c.col_date,
            tbl.c.col_group,
            sql.func.sum(tbl.c.col_summ)
        ).where(tbl.c.col_date == date)\
        .group_by(tbl.c.col_group, tbl.c.col_date)

        responce = self.conn.execute(responce).fetchall()
    except exc.SQLAlchemyError as err:
        transaction.rollback()
        print(err)
    else:
        return [
            (
                i[0].strftime('%d.%m.%Y'),
                i[1],
                f'{i[2]:,g}'.replace(',', ' ')
            ) for i in responce
        ]
    finally:
        transaction.close()
```
<br>Результать работы метода:

In [19]:
[print(' '.join([i[0], i[1].capitalize(), i[2]])) for i in db.calc_costs('12.10.2021')]

12.10.2021 Кофе 323.44
12.10.2021 Бензин 642
12.10.2021 Канцелярия 300


[None, None, None]

<p style="font-size: 25px">&#10077;</p>
<br><i>Вторая функция должна рассчитывать траты по всем группа в рамках периода.
<br>То есть, функция должна принимать две даты и возвращать вам агрегированные суммы трат по группа в рамках периода</i>
<p style="font-size: 25px">&#10078;</p>

Для решения задачи по рассчету трат по всем группам за период был написан метод calc_costs_between класса DataBase:
```python
def calc_costs_between(self, date_first: str, date_last: str):
    try:
        self.__metadata.reflect()
        tbl = self.__metadata.tables['costs']
        date_first = self.__check_date(date_first)
        date_last = self.__check_date(date_last)

        transaction = self.conn.begin()
        responce = select(
            tbl.c.col_group,
            sql.func.sum(tbl.c.col_summ)
        ).where(tbl.c.col_date.between(date_first, date_last))\
        .group_by(tbl.c.col_group)
        responce = self.conn.execute(responce).fetchall()
    except exc.SQLAlchemyError as err:
        transaction.rollback()
        print(err)
    else:
        return [
            (
                i[0],
                f'{i[1]:,g}'.replace(',', ' ')
            ) for i in responce
        ]
    finally:
        transaction.close()
```
<br>Результат работы метода:

In [20]:
[print(*i) for i in db.calc_costs_between('12.10.2021', '14.10.2021')]

кофе 805.54
бензин 742
канцелярия 1 023.09


[None, None, None]

#### Задание 2
<p style="font-size: 25px">&#10077;</p>
<i><br>Необходимо создать функцию, которая сможет удалить последнюю запись в таблице
<p style="font-size: 25px">&#10078;</p>

#### Решение задания 2

Для удаления последней записи в таблице воспользуемся методом del_last_rec класса DataBase:
```python
def del_last_rec(self):
    try:
        transaction = self.conn.begin()
        subquery = select(sql.func.max(self.tbl_costs.c.id))
        delete_ = delete(self.tbl_costs).where(
            self.tbl_costs.c.id == subquery.scalar_subquery()
        )
        self.conn.execute(delete_)
        print('Последняя запись удалена')
    except exc.SQLAlchemyError as err:
        print(err)
        transaction.rollback()
    else:
        transaction.commit()
    finally:
        transaction.close()
```

Содержимое таблицы 'costs' до использования метода del_last_rec

In [21]:
db.select_data('costs')

[(1, datetime.date(2021, 10, 12), 'канцелярия', 300.0),
 (2, datetime.date(2021, 10, 12), 'кофе', 123.45),
 (3, datetime.date(2021, 10, 12), 'кофе', 99.99),
 (4, datetime.date(2021, 10, 12), 'бензин', 642.0),
 (5, datetime.date(2021, 10, 12), 'кофе', 100.0),
 (6, datetime.date(2021, 10, 13), 'канцелярия', 200.0),
 (7, datetime.date(2021, 10, 13), 'кофе', 432.1),
 (8, datetime.date(2021, 10, 13), 'бензин', 100.0),
 (9, datetime.date(2021, 10, 14), 'канцелярия', 500.0),
 (10, datetime.date(2021, 10, 14), 'кофе', 50.0),
 (11, datetime.date(2021, 10, 14), 'канцелярия', 23.09)]

Удалим последнюю запись

In [22]:
db.del_last_rec()

Последняя запись удалена


Содержимое таблицы 'costs' после использования метода del_last_rec

In [23]:
db.select_data('costs')

[(1, datetime.date(2021, 10, 12), 'канцелярия', 300.0),
 (2, datetime.date(2021, 10, 12), 'кофе', 123.45),
 (3, datetime.date(2021, 10, 12), 'кофе', 99.99),
 (4, datetime.date(2021, 10, 12), 'бензин', 642.0),
 (5, datetime.date(2021, 10, 12), 'кофе', 100.0),
 (6, datetime.date(2021, 10, 13), 'канцелярия', 200.0),
 (7, datetime.date(2021, 10, 13), 'кофе', 432.1),
 (8, datetime.date(2021, 10, 13), 'бензин', 100.0),
 (9, datetime.date(2021, 10, 14), 'канцелярия', 500.0),
 (10, datetime.date(2021, 10, 14), 'кофе', 50.0)]

#### Задание 3

<p style="font-size: 25px">&#10077;</p><i><br>Необходимо создать функцию, которая предупредит вас о том, что вы потратили больше, чем нужно в рамках Месяца
<br> Предположим, у вас есть бюджет в 5 000 рублей, который направлен на развлечения или ваши увлечения
<br> Если вы потратите больше, чем 5 000 на конкретную группу трат, то функция должна будет вас об этом предупредить сообщением:
<br> "Вы вышли за пределы лимита в 5 000 рублей"></i>
<p style="font-size: 25px">&#10078;</p>

#### Решение задания 3

<br>Для проверки лимитов используется метод check_limit класса DataBase.
<br>Выполняется при каждом вызове метода insert_data - будь то запись одной или нескольких строк.
```python
def check_limit(self, month: 'datetime', group: str):
    try:
        transaction = self.conn.begin()

        query = select(sql.func.sum(self.tbl_costs.c.col_summ)).where(
            text(
                " TRUNC(col_date, 'MONTH') ="
                " TRUNC(:col_date, 'MONTH')"
                " AND col_group = :col_group"
            )
        ).group_by(text("TRUNC(col_date, 'MONTH'), col_group"))
        sum_costs = self.conn.execute(
            query,
            col_date=month,
            col_group=group
        ).fetchone()[0]

        query2 = select([self.tbl_costs_limits.c.col_limit]).where(
            text(
                "col_date = TRUNC(:col_date, 'MONTH')"
                "AND col_group = :col_group"
            )
        )

        query3 = query.union(query2)
        responce = self.conn.execute(
            query3,
            col_date=month,
            col_group=group
        ).fetchall()

        if responce[0][0] and (responce[1][0] > responce[0][0]):
            print(
                f"Вы вышли за пределы лимита в {responce[0][0]:,g} рублей"
                .replace(',', ' ')
        )
    except exc.SQLAlchemyError as er:
        print(er)
        transaction.rollback()
    except (IndexError, TypeError):
        pass
    finally:
        transaction.close()
```
<br>При множественном insert при превышении лимита по конкретной группе трат имеет смысл по каждой такой записи делать оповещение следующего вида:
<br>
```python
    "Книги 1300 - превышен лимит в 4000 рублей"
```
<br>Однако, это выходит за рамки поставленной задачи.

In [24]:
# установим лимит
db.set_limit('октябрь 2021', 'канцелярия', 3000)

# проверка достижения лимита
db.insert_tbl_costs(['12.10.2021', 'Канцелярия', 2200])


Вы вышли за пределы лимита в 3 000 рублей
